In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, score as sc

2020-12-22 15:54:46,415 - NumExpr defaulting to 8 threads.


In [2]:
path = '../raw_data/'

In [3]:
at_raw = pd.read_csv(path+'active_totes_20201210.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-0912.csv',names = ["timestamp","Pick Station","Availability","Blue Tote Loss","Grey Tote Loss"])
scs_raw = pd.read_csv(path + 'Faults20_11-10_12.csv')

In [4]:
# # HOTFIX - remove odd day
# at_raw = at_raw[~at_raw['DAY'].isin([9])]

In [5]:
fa = feat.add_code(scs_raw)
fa, unmapped = feat.add_tote_colour(fa)

Running: Function "load_tote_lookup" (id=2317012630256) was called 1 times


In [6]:
at = feat.pre_process_AT(at_raw)

Running: Function "pre_process_AT" (id=2317012631120) was called 1 times


In [7]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=2317012631408) was called 1 times


In [8]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=2317012629680) was called 1 times
Running: Function "load_module_lookup" (id=2317012628384) was called 1 times
Running: Function "load_ID_lookup" (id=2317012629104) was called 1 times
duplicated location faults removed - max duration kept
HOTFIX: Quadrant only faults, PTT Asset Code update


In [9]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

Time shifted by 0Minutes


In [10]:
mod = [2]
limit = 0

In [11]:
fa_sel_1 = feat.get_data_faults(fa_floor, modules = mod)

most_common = list(((fa_sel_1['Asset Code'].value_counts() > limit)[fa_sel_1['Asset Code'].value_counts() > limit]).index)
fa_sel = fa_sel_1[fa_sel_1['Asset Code'].isin(most_common)]

Running: Function "load_PTT_lookup" (id=2317012629968) was called 1 times


In [12]:
#fa_sel = feat.fault_select(fa_floor, fault_select_options = {'Tote Colour':['Blue']})
                                                            
fa_agg = feat.faults_aggregate(fa_sel, fault_agg_level= 'Asset Code', agg_type = 'count')

In [13]:
av_sel, at_sel = feat.av_at_select(av, at, remove_high_AT = True, availability_select_options = {"Module" : mod})

av_agg = feat.aggregate_availability(av_sel, agg_level = 'Module')
at_agg = feat.aggregate_totes(at_sel, agg_level = 'Module')

Running: Function "aggregate_availability" (id=2317012630112) was called 1 times


In [14]:
df = feat.merge_av_fa_at(av_agg ,at_df=at_agg, fa_df = fa_agg, agg_level = 'None')

In [15]:
X,y = md.gen_feat_var(df)
X = X.drop('Module', axis=1)
# Flip to availability
y=1-y

In [16]:
X_train, X_test, y_train, y_test = md.split(X[['TOTES']],y)

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

cv_R2 = md.cross_validate_r2(Linear_mdl, X[['TOTES']], y, n_folds = 5, shuffle = True, random_state = 101)


Cross Validation Scores LinearRegression(): 
 
        R2 Scores
1       0.428203
2       0.388557
3       0.304027
4       0.333328
5       0.387222
Mean    0.368267
STD     0.044070


In [17]:
X_train, X_test, y_train, y_test = md.split(X[['TOTES']],y)

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test,
                                                              fit_intercept=False)

cv_R2 = md.cross_validate_r2(Linear_mdl, X[['TOTES']], y, n_folds = 5, shuffle = True, random_state = 101)


Cross Validation Scores LinearRegression(fit_intercept=False): 
 
        R2 Scores
1      -0.026695
2      -0.343826
3      -0.552567
4      -0.400476
5      -0.362462
Mean   -0.337205
STD     0.171745


In [18]:
X_train, X_test, y_train, y_test = md.split(X,y)

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test,
                                                              fit_intercept=False)

cv_R2 = md.cross_validate_r2(Linear_mdl, X, y, n_folds = 5, shuffle = True, random_state = 101)


Cross Validation Scores LinearRegression(fit_intercept=False): 
 
        R2 Scores
1      -1.268416
2      -5.918697
3      -1.552172
4      -0.386447
5      -1.238571
Mean   -2.072861
STD     1.962077


In [19]:
# model = sm.OLS(y_train, X_train)
# results = model.fit()
# print(results.summary())

In [20]:
X_train_rd = X_train
max_p = 1
while max_p > 0.10:
    model = sm.OLS(y_train, X_train_rd)
    results = model.fit()
    max_p = max(results.pvalues)
    rm_col = results.pvalues[results.pvalues == max_p].index.values[0]
    X_train_rd = X_train_rd.drop([rm_col], axis=1)
X_train_rd.columns

C:\Users\chris.hughes\Anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\chris.hughes\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


Index(['C0503STA226', 'C06', 'C0602ACH025', 'C0602STA024', 'C0602STA030',
       'C0602STA049', 'C0602STA051', 'C0602STA058', 'C0602STA081',
       'C0603STA120', 'C06PTT021', 'C1501BCL231', 'C1502ACL069', 'C1502STA006',
       'C1502STA023', 'C1502STA034', 'C1502STA038', 'C1502STA046',
       'C1502STA237', 'C1502STA245', 'C1603BCL016', 'C1603BCL018',
       'C1603BCL054', 'C1603BCL056', 'C1603RDC035', 'C1603RDC125',
       'C1603RDC173', 'C1603STA103', 'C1603STA253', 'C2301ACH090',
       'C2301ACL001', 'C2301RDC025', 'C2301RDC061', 'C2301STA002',
       'C2301STA022', 'C2301STA086', 'C2302BCX093', 'C2302RDC015',
       'C2302RDC034', 'C2302RDC071', 'C2302STA014', 'C2303ACL016',
       'C2303STA042', 'C2303STA080', 'SCS021', 'SCS025', 'TOTES'],
      dtype='object')

In [21]:
model = sm.OLS(y_train, X_train_rd)
results = model.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:               Downtime   R-squared (uncentered):                   0.962
Model:                            OLS   Adj. R-squared (uncentered):              0.956
Method:                 Least Squares   F-statistic:                              152.9
Date:                Tue, 22 Dec 2020   Prob (F-statistic):                   7.38e-174
Time:                        15:56:28   Log-Likelihood:                          200.44
No. Observations:                 328   AIC:                                     -306.9
Df Residuals:                     281   BIC:                                     -128.6
Df Model:                          47                                                  
Covariance Type:            nonrobust                                                  
                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------

In [ ]:
filt_columns = X_train_rd.columns
X_train, X_test, y_train, y_test = md.split(X[filt_columns],y)

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test,
                                                              fit_intercept=False)

cv_R2 = md.cross_validate_r2(Linear_mdl, X[filt_columns], y, n_folds = 5, shuffle = True, random_state = 101)

In [ ]:
X_train_rd.drop([(results.pvalues == max(results.pvalues)).index.values], axis=1)

In [ ]:
keep_features = results.pvalues[results.pvalues < 0.10].index

In [ ]:
model = sm.OLS(y,X[keep_features])
results = model.fit()
print(results.summary())

In [ ]:
negs = results.params[results.params < 0].index
negs

In [ ]:
df['timestamp'].min()

In [ ]:
list(negs.values)

In [ ]:
fa[fa['Asset Code'].isin(list(negs.values))]

In [ ]:
fa[fa['Asset Code'].isin([negs]) & (fa['timestamp'] >= df['timestamp'].min())]#['Fault ID'].value_counts()